In [3]:
from datasets import load_dataset
from transformers import LlamaTokenizer

/gpfs/space/home/rodion/ttm/llama/venv_llama/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
def get_custom_dataset(dataset_config, tokenizer, split='train'):
    dataset = load_dataset('csv', data_files='energy_pandas.csv')

    prompt = (
        f"Convert the question into an SQL parse. Question:\n{{question}}\n---\nSQL parse:\n"
    )

    def apply_prompt_template(sample):
        return {
            "question": prompt.format(question=sample["natural_language"]),
            "parse": sample["parsed_utterance"],
        }

    dataset = dataset.map(apply_prompt_template)

    def tokenize_add_label(sample):
        question = tokenizer.encode(tokenizer.bos_token + sample["question"], add_special_tokens=False)
        parse = tokenizer.encode(sample["parse"] +  tokenizer.eos_token, add_special_tokens=False)

        sample = {
            "input_ids": question + parse,
            "attention_mask" : [1] * (len(question) + len(parse)),
            "labels": [-100] * len(question) + parse,
            }

        return sample

    dataset = dataset.map(tokenize_add_label)

    dataset = dataset['train'].train_test_split(test_size=0.2, seed=42)

    return dataset[split]

In [5]:
tokenizer = LlamaTokenizer.from_pretrained('meta-llama/Llama-2-7b-chat-hf')

In [18]:
dset = get_custom_dataset(None, tokenizer, 'test')

In [19]:
dset

Dataset({
    features: ['id', 'natural_language', 'parsed_utterance', 'question', 'parse', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 2673
})